In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests 
import json

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [219]:
class DCFModel:
    def __init__(self, ticker, apikey, number_of_years, risk_free_rate, discount_rate=0.1, margin_of_safety=0):
        self.ticker = ticker
        self.apikey = apikey
        self.number_of_years = number_of_years
        self.risk_free_rate = risk_free_rate
        self.discount_rate = discount_rate
        self.margin_of_safety = margin_of_safety
        self.fs = self.get_financials()
        self.ffs = self.get_ffinancials()
        self.effective_taxrate = None
        self.nopat = None
        self.operating_income_ratio = None
        self.fcff = None
        self.roic = None
        self.reinvestment_rate = None
        self.reinv_rate_percentage_sales = None
        self.reinv_rate_percentage_nopat = None
        self.salestocapital = None
        self.operating_income = None
        self.dcf_projections = None
        self.terminal_discount_rate = None
        self.terminal_roic = None
        self.terminal_reinvestment_rate = None
        self.terminal_value = None
        self.sum_of_npv = None
        self.intrinsic_value = None   
        self.intrinsic_value_per_share = None
        self.buy_price = None
        self.estimate()

    def get_financials(self):
        bs = self.get_json(self.ticker, self.apikey, statement="bs")
        iss = self.get_json(self.ticker, self.apikey, statement="is")
        cfs = self.get_json(self.ticker, self.apikey, statement="cfs")
        
        bs = pd.DataFrame(bs)
        bs = bs.set_index('date')
        
        iss = pd.DataFrame(iss)
        iss = iss.set_index('date')
        
        cfs = pd.DataFrame(cfs)
        cfs = cfs.set_index('date')
        
        fs = pd.concat([iss, bs, cfs], axis=1)
        fs = fs.loc[:,~fs.columns.duplicated()].copy() # remove duplicate columns
        fs = fs.iloc[::-1]

        return fs

    def get_ffinancials(self):
        fr = requests.get(f"https://financialmodelingprep.com/api/v3/financial-statement-full-as-reported/AAPL?period=annual&limit=50&apikey={self.apikey}")
        ffs = pd.DataFrame(fr.json())
        ffs = ffs.set_index('date')
        ffs = ffs.iloc[::-1]

        return ffs

    def est_efft(self):
        self.fs['efft'] = self.fs['incomeTaxExpense'] / self.fs['incomeBeforeTax'] 
        d = {'Historical': self.fs['efft'], 'Last' : self.fs['efft'].iloc[-1], 'Mean' : self.fs['efft'].mean()} 
        return pd.DataFrame(d)

    def est_nopat(self):
        self.fs['NOPAT'] = self.fs['operatingIncome']*(1-self.fs['efft'])
        d = {'Historical': self.fs['NOPAT'], 'Last' : self.fs['NOPAT'].iloc[-1], 'Mean' : self.fs['NOPAT'].mean()} 
        d1 = {'Historical': self.fs['operatingIncomeRatio'], 'Last' : self.fs['operatingIncomeRatio'].iloc[-1], 'Mean' : self.fs['operatingIncomeRatio'].mean()}
        return [pd.DataFrame(d), pd.DataFrame(d1)]

    def est_reinvss(self, exp_growth): 
        """Estimate investments as a percentage of sales using different methods to estimate average values of five years
           Estimate non-cash working capital based on revenue and thus, changes in non-cash WC are linked to revenue growth
        """
        # USELESS as of NOW. Work in Progress
        fs = self.fs
        fs['opWC as a % of Sales'] = fs['opWC'] / fs['revenue']
        avg_opWCs = fs['opWC as a % of Sales'].mean()
        avg_opWCs2 = fs['opWC'].sum() / fs['revenue'].sum()
        fs['capex as a % of Sales'] = (-fs['investmentsInPropertyPlantAndEquipment'] + fs['acquisitionsNet']) / fs['revenue']
        avg_capexs = fs['capex as a % of Sales'].mean()
        avg_capexs2 = (-fs['investmentsInPropertyPlantAndEquipment'].sum() + fs['acquisitionsNet'].sum()) / fs['revenue'].sum()
        fs['depreciation as a % of sales'] = fs['depreciationAndAmortization'] / fs['revenue']
        avg_dep = fs['depreciation as a % of sales'].mean()
        avg_dep2 = fs['depreciationAndAmortization'].sum() / fs['revenue'].sum()

        avgDelta_opWCs = avg_opWCs * exp_growth
        avgDelta_opWCs2 = avg_opWCs2 * exp_growth

        reinv_perc_sales = avg_capexs - avg_dep + avgDelta_opWCs
        self.reinv_rate_percentage_sales = reinv_perc_sales

        return reinv_perc_sales
        

    def est_fcff(self):
        fs = self.fs
        fs['opCA'] = fs['totalCurrentAssets'] - fs['cashAndShortTermInvestments']
        fs['opCL'] = fs['totalCurrentLiabilities'] - fs['shortTermDebt']
        fs['opWC'] = fs['opCA'] - fs['opCL']
        fs['deltaopWC'] = fs['opWC'] - fs['opWC'].shift(1) 
        fs['FCFF'] = fs['NOPAT'] + fs['depreciationAndAmortization'] - (-fs['investmentsInPropertyPlantAndEquipment'] - fs['acquisitionsNet']) - fs['deltaopWC']
        d = {'Historical': fs['FCFF'], 'Last' : fs['FCFF'].iloc[-1], 'Mean' : fs['FCFF'].mean()} 
        
        return pd.DataFrame(d)

    def est_roic(self):
        """
        Compute ROIC using two different methods, depending on what is the value of invested capital
        First definition of invested capital : https://www.youtube.com/watch?v=fRNcP9xjk-8&ab_channel=AswathDamodaran
        Second Method
        """
        fs = self.fs
        fs['investedcapital1'] = (fs['totalDebt'].shift(1) + fs["totalEquity"].shift(1) - fs['cashAndShortTermInvestments'].shift(1))
        fs['ROIC1'] = fs['operatingIncome']*(1-fs['efft']) / fs['investedcapital1']

        fs['investedcapital2'] = fs['opWC'] + fs['propertyPlantEquipmentNet'] + fs['goodwillAndIntangibleAssets'] + fs['longTermInvestments'] + fs['taxAssets']
        fs['ROIC2'] = fs['operatingIncome']*(1-fs['efft']) / fs['investedcapital2'].shift(1)
        
        df = pd.concat([fs['ROIC1'], fs['ROIC2']], axis=1)
        l = []
        for col in df.columns:
            d = {f'Historical_{col}': fs[col], f'Last_{col}' : fs[col].iloc[-1], f'Mean_{col}' : fs[col].mean()}
            l.append(pd.DataFrame(d))
            
        return pd.concat(l, axis=1)

    def est_reinvestments_rate(self):
        fs = self.fs
        fs['reinvestments'] = (-fs['investmentsInPropertyPlantAndEquipment'] - fs['acquisitionsNet'] + fs['deltaopWC'] - fs['depreciationAndAmortization']) 
        fs['reinvestment_rate'] = fs['reinvestments'] / (fs['operatingIncome']*(1-fs['efft']))
        
        d = {'Historical': fs['reinvestment_rate'], 'Last' : fs['reinvestment_rate'].iloc[-1], 'Mean' : fs['reinvestment_rate'].mean()}
        return pd.DataFrame(d)

    def est_salestocapital(self):
        """
        Estimate sales to capital as sales / invested capital.
        Useful metric to estimate the intrinsic growth of a company, especially of an unprofitable one. 
        """
        fs = self.fs
        fs['salestocapital1'] = fs['revenue'] / fs['investedcapital1']
        fs['salestocapital2'] = fs['revenue'] / fs['investedcapital2']

        df = pd.concat([fs['salestocapital1'], fs['salestocapital2']], axis=1)
        l = []
        for col in df.columns:
            d = {f'Historical_{col}': fs[col], f'Last_{col}' : fs[col].iloc[-1], f'Mean_{col}' : fs[col].mean()}
            l.append(pd.DataFrame(d))
            
        return pd.concat(l, axis=1) 
        
    def estimate(self):
        """ Estimate different measures crucial for the Discounted Cash flow model 
        The resulting estimates can be found in the instance attributes and, for each variable, they will be consisting of  
        - Historical estimate calculated each year
        - Last estimate at year t
        - 5 Year average of estimates
        """
        self.effective_taxrate = self.est_efft()
        [self.nopat, self.operating_income_ratio] = self.est_nopat()
        self.fcff = self.est_fcff()
        self.reinvestment_rate = self.est_reinvestments_rate()
        self.roic = self.est_roic()
        self.salestocapital = self.est_salestocapital()
        self.operating_income = pd.concat([self.fs['operatingIncome'], self.fs['operatingIncomeRatio']], axis=1)

    def create_projections(self, op_inc=None, fcff=None, nopat = None, roic=None, reinvrate=None):
        """Set self.dcf_projections
        Decide whether to input manually values (expect for discount rate and number of years which are already input manually in the init function)
        """
        dr = self.discount_rate
        ny = self.number_of_years
        nopat = nopat if nopat else self.nopat['Last'].unique()[0]
        fcf = fcff if fcff else self.fcff['Last'].unique()[0]
        r = roic if roic else self.roic['Mean_ROIC1'].unique()[0]
        rr = reinvrate if reinvrate else self.reinvestment_rate['Mean'].unique()[0]
        op_inc = op_inc if op_inc else self.operating_income['operatingIncome'].iloc[-1]
        self.dcf_projections = self.setup_dcf(op_inc, nopat, fcf, r, rr, dr, ny)

    def terminalvalue(self, t_roic, t_g, t_dr, t_efft):
        """
        terminalvalue with terminal_roic, terminal growth, terminal discount rate and terminal effective tax rate as input
        """
        t_rfr = self.risk_free_rate
        op_inc = self.dcf_projections['operatingIncome']
        [self.terminal_cashflow, self.terminal_value] = self.terminal_value_formula(op_inc, t_roic, t_g, t_dr, t_efft, t_rfr)
        return [self.terminal_cashflow, self.terminal_value]

    def npv(self):
        fcf_npv = self.dcf_projections['NPV FCF'].sum()
        tv_dfactor = 1/(1 + self.discount_rate)**(self.number_of_years+1) # terminal value discount factor
        tv_npv = self.terminal_value * tv_dfactor
        npv = fcf_npv + tv_npv
        self.sum_of_npv = npv
        
        return npv

    def valuation(self):
        """Function to find the intrinsic value of the company according to FCFF valuation
        terminalvalue function has to be called before performing this one, otherwise terminal_value is assumed to be 0
        """
        if not self.terminal_value:
            self.terminal_value = 0
        else:
            pass
        npv = self.npv()
        debt = self.fs['totalDebt'].iloc[-1]
        min_interest = self.fs['minorityInterest'].iloc[-1]
        cash = self.fs['cashAndShortTermInvestments'].iloc[-1]
        self.intrinsic_value = npv - debt - min_interest + cash
        self.intrinsic_value_per_share = self.intrinsic_value / self.fs['weightedAverageShsOut'].iloc[-1]
        self.buy_price = self.intrinsic_value_per_share * (1 - self.margin_of_safety)

        return f"The intrinsic value of {self.ticker} is {self.intrinsic_value}, with a value per share of {self.intrinsic_value_per_share}. Considering a margin of safety of {self.margin_of_safety}, the buy price for the stock is {self.buy_price}"
        
    def factor_analysis(self):
        pass
    
    @staticmethod
    def get_json(ticker, apikey, statement = "is", period="annual"):
        """ Get financial statements in json format using financialmodelingprep api
    
        :param str statement can be is (income statement), bs (balance sheet), "cfs (cashflow statement)"
        :param str period can be annual or quarterly
        """
        which_statement = {"is" : "income-statement", 
                           "bs" : "balance-sheet-statement", 
                           "cfs" : "cash-flow-statement"}
        fst = which_statement[f"{statement}"]
        url = "https://financialmodelingprep.com/api/v3/{s}/{t}?period={p}&apikey={a}".format(t = ticker, s = fst, p = period, a = apikey)
        r = requests.get(url)
    
        return r.json()

    @staticmethod
    def setup_dcf(operating_income, nopat, fcf, roic, rr, dr, ny=5, start='t'):
        """
        Forecast cash flow to firm based on the growth rate given by ROIC and Reinvestments.
        :param float Free Cash Flow base year (t=0)
        :param float roic return on invested capital
        :param float rr retention rate
        :param float dr discount rate 
        :param int ny Number of years to project. If none is 5
        :param str start Decide when to start base year, 't' or 't1'. If none is 't'
        """
        exp_g = roic * rr
        op_inc = [operating_income]
        npt = [nopat]
        yrs = ['BaseYear']
        dfactor = [1]
        for i in range(ny):
            op_inc.append((1+exp_g) * op_inc[i])
            npt.append((1+exp_g) * npt[i])
            yrs.append(f'Year {str(i+1)}')
            dfactor.append(1/(1+dr)**(i+1))
    
        data = {'operatingIncome': op_inc, 'NOPAT': npt}
    
        df = pd.DataFrame(data, index=yrs)
        df['FCF'] = df['NOPAT'] * (1 - rr)
        df['Discount Factor'] = dfactor
        df['NPV FCF'] = df['FCF'] * df['Discount Factor']
        
        return df

    @staticmethod
    def terminal_value_formula(operating_income, t_roic, t_g, t_dr, t_efft, t_rfr):
        """
        Going concern / stable growth model terminal value from operating income at n (last year of projections of growth period)
        """
        t_g = t_rfr if t_g >= t_rfr else t_g
        t_rr =  t_g / t_roic
        terminal_cashflow = operating_income.iloc[-1]*(1-t_efft)*(1-t_rr)*(1+t_g)
        terminal_value = terminal_cashflow / (t_dr - t_g)
        return [terminal_cashflow, terminal_value]

    @staticmethod
    def capitalize_rd(rd, p=5):
        """
        R&D according to accounting are treated as expenses, but they should be capitalized to better reflect past and future profitability
        assume constant depreciation expense from R&D (which becomes a capital expense)
        :param rd_exp pd.Series of historical R&D e.g. rd = META.fs['researchAndDevelopmentExpenses']
        :param period int number of years intended as amortization period (maximum 5 years because data from FMP free api comprise only 5 years
        """
        ps = pd.Series(range(p))

        rd = pd.DataFrame({'RD_exp' : rd, 'p': ps.values})

        rd['amortization'] = rd['RD_exp'] / p

        rd['bv'] = rd['RD_exp'] * rd['p'] / p

        total_rd_bv = rd['bv'].sum() # Total book value of R&D at t

        total_rd_amortization = rd['amortization'].sum() # Total R&D expenses from t-1 to t 

        return rd, total_rd_bv, total_rd_amortization


    

In [221]:
help(DCFModel.terminalvalue)

Help on function terminalvalue in module __main__:

terminalvalue(self, t_roic, t_g, t_dr, t_efft)
    terminalvalue with terminal_roic, terminal growth, terminal discount rate and terminal effective tax rate as input



In [223]:
help(DCFModel.create_projections)

Help on function create_projections in module __main__:

create_projections(self, op_inc=None, fcff=None, nopat=None, roic=None, reinvrate=None)
    Set self.dcf_projections
    Decide whether to input manually values (expect for discount rate and number of years which are already input manually in the init function)



In [225]:
myapikey = ""
ticker = "META"
number_of_years = 10
rfr = 0.03 # riskfreerate
mos = 0.3 # margin of safety

In [227]:
META = DCFModel(ticker, myapikey, number_of_years, rfr)
META.create_projections()
META.terminalvalue(0.15, rfr, 0.1, 0.25)
META.valuation()

'The intrinsic value of META is 1359996390126.6655, with a value per share of 528.3591259233355. Considering a margin of safety of 0, the buy price for the stock is 528.3591259233355'

In [228]:
META.dcf_projections

,operatingIncome,NOPAT,FCF,Discount Factor,NPV FCF
BaseYear,50203000000.000,41385613856.793,21858714482.931,1.000,21858714482.931
Year 1,60163219838.139,49596474007.581,26195459331.428,0.909,23814053937.662
Year 2,72099536308.434,59436359757.678,31392609575.479,0.826,25944305434.280
Year 3,86404004803.542,71228467989.560,37620868696.739,0.751,28265115474.635
Year 4,103546464073.683,85360117490.783,45084807559.383,0.683,30793530195.603
Year 5,124089968359.003,102295470669.231,54029583661.442,0.621,33548120571.398
Year 6,148709281239.975,122590779242.653,64748993478.208,0.564,36549118815.670
Year 7,178213038647.338,146912654656.186,77595122381.684,0.513,39818566985.266
Year 8,213570308988.758,176059963330.574,92989909093.412,0.467,43380478877.109
Year 9,255942422775.332,210990065903.473,111439004511.992,0.424,47261016407.336
